In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore') #Importing the basic libraries

In [ ]:
df=pd.read_csv("../input/housesalesprediction/kc_house_data.csv")
df.head()

In [ ]:
df.shape

# **Basic Cleaning of Data**

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum().sum()

In [ ]:
df=df.drop(['id', 'date','zipcode'],axis=1)
#We can drop the above 3 columns as it will irrevelent in predicting the price of houses.

In [ ]:
df.columns

In [ ]:
#lets go through the above columns and decide if we want to put them into either category or numerical.

In [ ]:
sns.distplot(df['bedrooms'])
plt.show()

In [ ]:
sns.distplot(df['bathrooms'])
plt.show()

In [ ]:
df['bathrooms'].value_counts()

In [ ]:
df['bathrooms']=round(df['bathrooms'])

In [ ]:
df['bathrooms'].value_counts()

In [ ]:
df['floors']=round(df['floors'])

In [ ]:
df['floors'].value_counts()

In [ ]:
df['floors']=df['floors'].astype('object')

In [ ]:
df['waterfront'].value_counts()

In [ ]:
df['waterfront']=df['waterfront'].astype('object')

In [ ]:
df['view'].value_counts()

In [ ]:
df['view']=df['view'].astype('object')

In [ ]:
df['condition'].value_counts()

In [ ]:
df['condition']=df['condition'].astype('object')

In [ ]:
df['grade'].value_counts()

In [ ]:
df['grade']=df['grade'].astype('object')

In [ ]:
df['yr_built'].value_counts()

In [ ]:
df['yr_renovated'].value_counts()

In [ ]:
df.info()

In [ ]:
df_cat=df.select_dtypes(include='object')
df_num=df.select_dtypes(exclude='object')

# **Exploratory data Analysis**

In [ ]:
plt.figure(figsize=[12,8])
sns.heatmap(df_num.corr(),annot=True)
plt.show()

Observation-
1. sqft_living15,sqft_above,sqft_living,bathrooms columns are having a very good correlation with the      price.
2. we can observe the multicollinearity in the data as input feature are having strong correlation        among them.
3. On the basis of correlation we can drop the columns like sqft_lot15, long,yr_built,sqft_lot.

In [ ]:
df_num.columns

In [ ]:
df_num_drop=df_num[['sqft_lot15','long','yr_built','sqft_lot']]

In [ ]:
df_num=df_num.drop(['sqft_lot15','long','yr_built','sqft_lot'],axis=1)

In [ ]:
distplot=df_num[['price', 'bedrooms','sqft_living','sqft_above', 'sqft_basement','lat',
    'sqft_living15']]
for i in distplot:
    sns.distplot(df_num[i])
    plt.show()

There are a lot of values which are zero in sqft_basement which we will have to impute.

In [ ]:
df_num[df_num['sqft_basement']==0].shape #We will have to drop this column as it has high number
                                         #zero values.

In [ ]:
df_num=df_num.drop('sqft_basement',axis=1)

In [ ]:
df_num=df_num.drop('yr_renovated',axis=1)

In [ ]:
for i in df_cat.columns:
    plt.figure(figsize=[10,4])
    sns.boxplot(x=df_cat[i],y=df_num['price'])
    plt.show()

Observation-
1. If we compare the number of floors with the price of house then it was found that mean salaries are 
   almost equal irrespective of the number of floors. however maximum number of houses which have          higher prices, have two floors.
2. The houses which have waterfront have high mean salaries as compare to the houses which don't have      waterfront. 
3. lets compare the number of view in a house with its price and it was found that higher the number of    view then higher the price, however the houses which have view 1 and 2, they are having same mean      price.
4. Condition type 3,4 and 5 are having almost equal mean price of the house, similarly condition type 1    and 2 have almost same mean price. The houses which have condition type 3 and 4 have higher price.
5. As per the observation higher the number of grade, higher the mean price of the house. we can say      price is directly propotional to the grade type.

In [ ]:
df_num.columns

In [ ]:
df_cat.columns

Top 10 houses with price

In [ ]:
top_10=df['price'].sort_values(ascending=False).head(10)
top_10.index=[1,2,3,4,5,6,7,8,9,10]
top_10=pd.DataFrame(top_10)
plt.figure(figsize=[10,4])
sns.barplot(x=top_10.index,y=top_10['price'])
plt.show()

In [ ]:
plt.figure(figsize=[10,4])
sns.scatterplot(x=df_num['sqft_living'],y=df_num['price'],hue=df_cat['condition'],palette='Set1')
plt.show()

As we saw earlier that price and sqft_living have strong correlation but if we compare it with the condition of house we can see that maximum number of houses are condition 3 and highest price of the house has condition 4. One interesting finding we can observe from the graph is that, the house which has maximum sqft_living has low price which is a exception. 

In [ ]:
plt.figure(figsize=[10,4])
sns.scatterplot(x=df_num['sqft_above'],y=df_num['price'],hue=df_cat['view'],palette='Set1')
plt.show()

Most of the houses have view type 0 and the house which has the highest price, have view type 3. we know that sqft_above is highly correlated with the price but the house which has the highest sqft_above has very low price.

In [ ]:
plt.figure(figsize=[10,4])
sns.scatterplot(x=df_num['lat'],y=df_num['price'],hue=df_cat['grade'],palette='Set1')
plt.show()

Most of the houses which have higher prices they have lat between 47.5 to 47.8 and the house which has highest price is of grade 13 and surprisingly 2nd highest price house is of grade 1.

# **Outlier Treatment**

In [ ]:
for i in df_num.columns:
    sns.boxplot(df_num[i])
    plt.show()

In [ ]:
for i in df_num.columns:
    q1=df_num[i].quantile(0.25)
    q3=df_num[i].quantile(0.75)
    iqr=q3-q1
    ub=q3+1.5*iqr
    lb=q1-1.5*iqr
    for j in range(len(df_num)):
        if df_num[i][j]>ub:
            df_num[i][j]=df_num[i].quantile(0.90)
        elif df_num[i][j]<lb:
            df_num[i][j]=df_num[i].quantile(0.1)
        else:
            pass

In [ ]:
for i in df_num.columns:
    sns.boxplot(df_num[i])
    plt.show()

# **Data Transformation**

In [ ]:
from sklearn.preprocessing import PowerTransformer
pt=PowerTransformer()
df_num_pt=pt.fit_transform(df_num)
df_num_pt=pd.DataFrame(df_num_pt)
df_num_pt.columns=df_num.columns
df_num_pt.head()

In [ ]:
df_cat.columns

In [ ]:
df_cat_dum=pd.get_dummies(df_cat,columns=['floors', 'waterfront', 'view', 'condition', 'grade'],drop_first=True)
df_cat_dum.head()

In [ ]:
final=pd.concat([df_num_pt,df_cat_dum],axis=1)
final.head()

In [ ]:
x=final.drop('price',axis=1)
y=final['price']

# **Base Model**

In [ ]:
import statsmodels.api as sm
inpc=sm.add_constant(x)
ols=sm.OLS(y,inpc)
ols_mod=ols.fit()
ols_mod.summary()

# **Model Assumptions**

In [ ]:
#1. Multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(x.values,i) for i in range(x.shape[1])]
vif['feature']=x.columns
vif.sort_values('VIF',ascending=False)

In [ ]:
x1=x.drop("condition_3",axis=1)
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(x1.values,i) for i in range(x1.shape[1])]
vif['feature']=x1.columns
vif.sort_values('VIF',ascending=False) #Now Vif is in acceptable range.

In [ ]:
#2.Linearity
ypred=ols_mod.predict()
sns.scatterplot(y,ypred)
plt.show()

In [ ]:
#3 Normality
res=ols_mod.resid
res.plot(kind='kde')
plt.show()

In [ ]:
#4. Auto Correlation is 1.9

In [ ]:
#5. Homoscadasticity
sns.residplot(ols_mod.predict(),ols_mod.resid)
plt.show()

# **Feature Selection**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x1,y,test_size=0.3,random_state=48)

In [ ]:
lr=LinearRegression()
back_mod=sfs(estimator=lr,k_features='best',forward=False,scoring='r2')
back_feat=back_mod.fit(xtrain,ytrain)
backward_feature=list(back_feat.k_feature_names_)
print(backward_feature)

In [ ]:
inp=x1[backward_feature]

# **Model Building**

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(inp,y,test_size=0.3,random_state=48)
lr=LinearRegression()
lr.fit(xtrain,ytrain)

In [ ]:
ypred_train=lr.predict(xtrain)
ypred_test=lr.predict(xtest)

rmse_train=np.sqrt(mean_squared_error(ytrain,ypred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,ypred_test))

r2_train=r2_score(ytrain,ypred_train)
r2_test=r2_score(ytest,ypred_test)

res_lr=[rmse_train,rmse_test,r2_train,r2_test]
res_lr

In [ ]:
models=pd.DataFrame()
models['LinearReg']=res_lr
models.index=['Bias_Error_RMSE','Var_Error_RMSE','Bias_Error_R2','Var_error_R2']
models

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt_model=DecisionTreeRegressor()
dt_model.fit(xtrain,ytrain)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameter={'max_depth':np.arange(1,30)}
GS=GridSearchCV(dt_model,parameter,cv=5,scoring='r2')
GS.fit(inp,y)

In [ ]:
GS.best_params_

In [ ]:
dt_model2=DecisionTreeRegressor(max_depth=9,random_state=0)
dt_model2.fit(xtrain,ytrain)

In [ ]:
ypred_train=dt_model2.predict(xtrain)
ypred_test=dt_model2.predict(xtest)

rmse_train=np.sqrt(mean_squared_error(ytrain,ypred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,ypred_test))

r2_train=r2_score(ytrain,ypred_train)
r2_test=r2_score(ytest,ypred_test)

res_lr=[rmse_train,rmse_test,r2_train,r2_test]
res_lr

In [ ]:
models['DecisionTreeReg']=res_lr

In [ ]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
RF_ve=[]
RF_be=[]
for n in np.arange(1,10):
    RF=RandomForestRegressor(n_estimators=n,random_state=0)
    kf=KFold(shuffle=True,n_splits=5,random_state=0)
    r2=cross_val_score(RF,inp,y,cv=kf,scoring='r2')
    RF_be.append(np.mean(1-r2))
    RF_ve.append(np.std(r2))

In [ ]:
np.min(RF_ve),np.argmin(RF_ve)

In [ ]:
rf=RandomForestRegressor(n_estimators=6,random_state=0)
rf.fit(xtrain,ytrain)

In [ ]:
ypred_train=rf.predict(xtrain)
ypred_test=rf.predict(xtest)

rmse_train=np.sqrt(mean_squared_error(ytrain,ypred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,ypred_test))

r2_train=r2_score(ytrain,ypred_train)
r2_test=r2_score(ytest,ypred_test)

res_lr=[rmse_train,rmse_test,r2_train,r2_test]
res_lr

In [ ]:
models['RandomForestReg']=res_lr

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn=KNeighborsRegressor()
param={'n_neighbors':np.arange(1,10)}
kf=KFold(shuffle=True,n_splits=5,random_state=0)
GS=GridSearchCV(knn,param,cv=kf,scoring='r2')
GS.fit(inp,y)

In [ ]:
GS.best_params_

In [ ]:
knn=KNeighborsRegressor(n_neighbors=50,weights='uniform')
knn.fit(xtrain,ytrain)

In [ ]:
ypred_train=knn.predict(xtrain)
ypred_test=knn.predict(xtest)

rmse_train=np.sqrt(mean_squared_error(ytrain,ypred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,ypred_test))

r2_train=r2_score(ytrain,ypred_train)
r2_test=r2_score(ytest,ypred_test)

res_lr=[rmse_train,rmse_test,r2_train,r2_test]
res_lr

In [ ]:
models['KNeighborsReg']=res_lr

In [ ]:
from sklearn.linear_model import Ridge,Lasso,ElasticNet

In [ ]:
rid=Ridge()
param={'alpha':[0.000001,0.00001,0.0001,0.001,0.01,0.1,0.5,1,3,5,10,50,100]}
grid=GridSearchCV(rid,param_grid=param,cv=5,scoring='neg_mean_squared_error')
hyp_rid=grid.fit(xtrain,ytrain)
hyp_rid.best_params_

In [ ]:
rid=Ridge(alpha=0.5)
rid.fit(xtrain,ytrain)

In [ ]:
ypred_train=rid.predict(xtrain)
ypred_test=rid.predict(xtest)

rmse_train=np.sqrt(mean_squared_error(ytrain,ypred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,ypred_test))

r2_train=r2_score(ytrain,ypred_train)
r2_test=r2_score(ytest,ypred_test)

res_lr=[rmse_train,rmse_test,r2_train,r2_test]
res_lr

In [ ]:
models['Ridge']=res_lr

In [ ]:
las=Lasso()
param={'alpha':[0.0001, 0.001, 0.1, 1, 2, 3]}
grid=GridSearchCV(las,param_grid=param,cv=5,scoring='neg_mean_squared_error')
hyp_rid=grid.fit(xtrain,ytrain)
hyp_rid.best_params_

In [ ]:
las=Lasso(alpha=0.0001)
las.fit(xtrain,ytrain)

In [ ]:
ypred_train=las.predict(xtrain)
ypred_test=las.predict(xtest)

rmse_train=np.sqrt(mean_squared_error(ytrain,ypred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,ypred_test))

r2_train=r2_score(ytrain,ypred_train)
r2_test=r2_score(ytest,ypred_test)

res_lr=[rmse_train,rmse_test,r2_train,r2_test]
res_lr

In [ ]:
models['lasso']=res_lr

In [ ]:
enet=ElasticNet()
param={'alpha':[0.000001,0.00001,0.0001,0.001,0.01,0.1,0.2,0.5,1,3,5,10,50,100],
      'l1_ratio':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}
grid=GridSearchCV(enet,param_grid=param,cv=kf,scoring='neg_mean_squared_error')
hyp_enet=grid.fit(xtrain,ytrain)
hyp_enet.best_params_

In [ ]:
enet=ElasticNet(alpha=0.00001,l1_ratio=0.1)
enet.fit(xtrain,ytrain)

In [ ]:
ypred_train=enet.predict(xtrain)
ypred_test=enet.predict(xtest)

rmse_train=np.sqrt(mean_squared_error(ytrain,ypred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,ypred_test))

r2_train=r2_score(ytrain,ypred_train)
r2_test=r2_score(ytest,ypred_test)

res_lr=[rmse_train,rmse_test,r2_train,r2_test]
res_lr

In [ ]:
models['ElasticNet']=res_lr

In [ ]:
models

# **Conclusion-**
If we Compare all the above models then we can observe that knn regressor is perfoming better with least bias variance error gap and it is able to explain 77.8% variance.